In [ ]:
import pandas as pd
import numpy as np
import os
import random
import glob

In [ ]:
os.environ['GGS_DIR'] = '/kaggle/input/gitaggle-settings/gitaggle-settings'
os.environ['USER_NAME'] = 'Dinh Thoai Tran @ randrise.com'
os.environ['USER_EMAIL'] = 'dinhtt@randrise.com'
os.environ['DT_REPO_URL'] = 'https://github.com/dinhtt-randrise/lottery-randrise.git'
os.environ['DEBUG_ON'] = 'no'
USER_NAME = os.environ.get('USER_NAME', 'Dinh Thoai Tran @ randrise.com')
USER_EMAIL = os.environ.get('USER_EMAIL', 'dinhtt@randrise.com')
DEBUG_ON = os.environ.get('DEBUG_ON', 'no') == 'yes'
DT_REPO_URL = os.environ.get('DT_REPO_URL', 'https://github.com/dinhtt-randrise/lottery-randrise.git')


In [ ]:
!git clone https://github.com/dinhtt-randrise/gitaggle.git

In [ ]:
import gitaggle.github as gggh

TODAY_FORCED = None
root_dir = '/kaggle/working/data'
lottery_company = 'valottery'
lottery_kind = 'powerball'

!mkdir -p $root_dir

folder_name = DT_REPO_URL.split('/')[-1].replace('.git', '')
repo_dir = root_dir + '/' + folder_name

gggh.clone(DT_REPO_URL, root_dir)

if TODAY_FORCED is None:
    files = glob.glob(repo_dir + '/' + lottery_company + '/' + lottery_kind + '/raw/' + lottery_kind + '-*.*.*.txt')
    df = pd.DataFrame({'filename': files})
    df = df.sort_values(by=['filename'], ascending=[False])
    for ri in range(len(df)):
        TODAY = df['filename'].iloc[ri].split('/')[-1].replace('.txt', '').replace(lottery_kind + '-', '')
        FILE = repo_dir + '/' + lottery_company + '/' + lottery_kind + '/raw/' + lottery_kind + '-' + TODAY + '.txt'
        CSV_FILE = repo_dir + '/' + lottery_company + '/' + lottery_kind + '/data/' + lottery_kind + '-' + TODAY + '.csv'
        if os.path.exists(FILE) and not os.path.exists(CSV_FILE):
            break
else:
    TODAY = TODAY_FORCED
FILE = repo_dir + '/' + lottery_company + '/' + lottery_kind + '/raw/' + lottery_kind + '-' + TODAY + '.txt'
CSV_FILE = repo_dir + '/' + lottery_company + '/' + lottery_kind + '/data/' + lottery_kind + '-' + TODAY + '.csv'


In [ ]:
def pad_2(s):
    t = ''
    v = int(s)
    if v < 10:
        t = '0' + str(v)
    else:
        t = str(v)
    return t

In [ ]:
lines = []
with open(FILE, 'r') as f:
    lines = f.readlines()
    
rows = []
for ln in lines:
    ln = ln.rstrip()
    fds = ln.split(';')
    if len(fds) < 3:
        continue
    if len(fds) > 3:
        continue
        
    date_str = fds[0].lstrip().rstrip()
    date_fds = date_str.split('/')
    date = date_fds[2] + '.' + pad_2(date_fds[0]) + '.' + pad_2(date_fds[1])

    s = fds[1].lstrip().rstrip()
    sl = s.split(',')
    for ni in range(len(sl)):
        no = ni + 1
        n = int(sl[ni])
        rw = {'date': date, 'hour': 'N' + str(no), 'n': n}
        rows.append(rw)

    s = fds[2].replace('Powerball: ', '').lstrip().rstrip()
    n = int(s)
    rw = {'date': date, 'hour': 'PB', 'n': n}
    rows.append(rw)
        
df = pd.DataFrame(rows)
df = df.sort_values(by=['date', 'hour'], ascending=[False, True])
df.to_csv('powerball-' + TODAY + '.csv', index=False)
df.to_csv(CSV_FILE, index=False)
df

In [ ]:
gggh.write_text_file('/kaggle/working/msg.txt', 'Import raw data @ ' + TODAY)
gggh.identity(repo_dir, USER_NAME, USER_EMAIL)
gggh.commit(repo_dir, '/kaggle/working/msg.txt')
!rm -rf $root_dir
!rm -rf /kaggle/working/msg.txt